# Практическая работа №2 Основы Python

## 1. Парсинг вакансий

In [ ]:
import requests as req
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json

In [ ]:
ua = UserAgent()
hdr = {"Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "User-Agent": ua.chrome}

In [ ]:
data_list = []

In [ ]:
data_dict = {'data':data_list}

In [ ]:
area = ['3&area=5', '&area=40', '&area=9', '&area=16', '&area=28', '&area=1001', '&area=48', '&area=97', '&area=113']
experience = ['doesNotMatter', 'between3And6', 'between1And3', 'moreThan6', 'noExperience']

# Выбираем регион поиска вакансий
for place in area:
    # Выбираем требуемый опыт работы
    for work_exp in experience:

        # Находим максимальное количество страниц, соответствующее запросу по ссылке
        page_iter = 1
        page_max = 0
        while page_iter != 0:
            url = f"https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=python+разработчик{place}&salary=&currency_code=RUR&experience={work_exp}&order_by=relevance&search_period=0&items_on_page=20&page={page_max}&hhtmFrom=vacancy_search_list"
            resp = req.get(url, headers=hdr)
            soup = BeautifulSoup(resp.text, 'lxml')
            next_button = soup.find(attrs={"data-qa":"pager-next"})
            try:
                button = str(next_button.text)
            except:
                button = 'null'
                  
            if button == 'дальше':
                page_max += 1
            else:
                page_iter = 0
            
        # Перебираем страницы
        for page in range(0, page_max+1):
            url = f"https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&text=python+разработчик{place}&salary=&currency_code=RUR&experience={work_exp}&order_by=relevance&search_period=0&items_on_page=20&page={page}&hhtmFrom=vacancy_search_list"
            resp = req.get(url, headers=hdr)
            soup = BeautifulSoup(resp.text, 'lxml')
            tags = soup.find_all(class_='serp-item__title')
            
            # Собираем со страниц с подробным описанием вакансий необходимую информацию
            for i in tags:
                resp_object = req.get(i.attrs['href'], headers=hdr)
                soup_object = BeautifulSoup(resp_object.text, 'lxml')
                tag_price = soup_object.find(class_='bloko-header-section-2 bloko-header-section-2_lite')
                tag_vacancy = soup_object.find(class_='vacancy-title')
                tag_exp = soup_object.find(attrs={'data-qa':"vacancy-experience"})
                tag_region = soup_object.find(attrs={'data-qa':"vacancy-view-link-location"})

                try:
                    region = tag_region.text
                    price = tag_price.text
                    vacancy = tag_vacancy.text
                    exp = tag_exp.text
                except:
                    region = "undefinite"
                    price = "undefinite"
                    vacancy = "undefinite"
                    exp = "undefinite"

                if region == 'undefinite':
                    tag_region = soup_object.find(attrs={'data-qa':"vacancy-view-location"})
                if vacancy == 'undefinite':
                    tag_vacancy = soup_object.find(class_='bloko-header-section-1')
                if price == 'undefinite':
                    tag_price = soup_object.find(class_='bloko-header-section-2 bloko-header-section-2_lite')
                if exp == 'undefinite':
                    tag_exp = soup_object.find(class_="vacancy-description-list-item")

                try:
                    region = tag_region.text
                except:
                    region = "undefinite"
                try:
                    vacancy = tag_vacancy.text
                except:
                    vacancy = "undefinite"
                try:
                    price = tag_price.text
                except:
                    price = "undefinite"
                try:
                    exp = tag_exp.text
                except:
                    exp = "undefinite"

                # Небольшая предобработка данных перед записью в json
                salary = str(price)
                salary = salary.replace('\xa0', ' ')
                vacancy = str(vacancy)
                vacancy = vacancy.replace('\xa0', ' ')
                vacancy = vacancy.replace(salary, '')
                region = region.split(',')
                region = str(region[0])
                exp = exp.replace('Требуемый опыт работы: ', '')

                vacancy_dict = {'title':vacancy,
                                'work experience':exp,
                                'salary':salary,
                                'region':region}
                
                if vacancy_dict not in data_list:
                    data_list.append(vacancy_dict)

                with open('parsed.json', 'w', encoding='utf8') as file:
                    json.dump(data_dict, file, ensure_ascii=False, indent=4, sort_keys=False)

## 2. Палиндром строки

In [ ]:
phrase = str(input())
p_list = list(phrase)
if ' ' in p_list:
    p_list.remove(' ')
for i in p_list: 
    if i.isupper() == True or i.isnumeric() == True:
        print('Допустимымы символами являются только пробелы и буквы в нижнем регистре')
        break
    else:
        r_list = list(reversed(p_list))
        r_list == p_list

## 3. Перевод арабского числа в римское

In [ ]:
def rome(number):
    result = ''
    numbers_dict = {1000:'M',
                    900:'CM',
                    500:'D',
                    400:'CD',
                    100:'C',
                    90:'XC',
                    50:'L',
                    40:'XL',
                    10:'X',
                    9:'IX',
                    5:'V',
                    4:'IV',
                    1:'I'}
    x = number
    if number < 0:
        number *= -1
    for arabic in numbers_dict:
        # Получаем значение ключа
        roman = numbers_dict[arabic]
        ''' 
        Делим исходное число без остатка и умножаем 
        полученное значение на на римский аналог.
        Записываем полученную комбинацию в переменную 
        result
        '''
        result += number // arabic * roman
        ''' 
        Делим исходное число по модулю и в следующей 
        итерации проделываем то же самое с остатком
        ''' 
        number %= arabic
    if x < 0:
        result = '-'+result
    return result
    

## 4. Валидность скобок 

In [ ]:
def bkt_valid(string):
    
    if string == '':
        string = 0
        
    value_list = list(str(string))
    bkt_list = ['{', '[', '(', ')', ']', '}']
    check = 1
    
    for i in value_list:
        if i not in bkt_list or i == 0:
            check = 0
            print('error')
        
    if check == 1:
        validator = 0
        for bkt in value_list:
            if bkt == '(' or bkt == '{' or bkt == '[':
                validator += 1
            if bkt == ')' or bkt == '}' or bkt == ']':
                validator -= 1
        if validator == 0:
            valid = True
        else:
            valid = False
        return valid

## 5. Умножить два бинарных числа в формате строк

In [ ]:
def binar_calc(x1, x2):
    check = 1
    error = "Аргументами могут быть только положительные числа, содержащие 0 или 1."
    num1 = 0
    num2 = 0
    
    y1 = list(reversed(list(str(x1))))
    y2 = list(reversed(list(str(x2)))) 

    for value in [y1, y2]:
        for i in value:
            if check == 1:
                if int(i) >= 0 and int(i) <= 1:
                    check = 1
                else:
                    check = 0
                    
    if check == 1:
        index_list_1 = [i for i, e in enumerate(y1) if int(e) != 0]
        index_list_2 = [i for i, e in enumerate(y2) if int(e) != 0]
        for index in index_list_1:
            num1 += 2**index
        for index in index_list_2:
            num2 += 2**index
        result = num1 * num2
        
        x = result
        multi_list = []
        l = x
        
        while x > 0:
            x = x // 2
            y = l - x*2
            l = x
            multi_list.append(str(y))
        multi_list = list(reversed(multi_list))
        multi_list = ''.join(multi_list)
        if result == 0:
            multi_list = '0'
        
        print(f'Первое число в десятичной системе: {num1}')
        print(f'Второе число в десятичной системе: {num2}')
        print(f'Произведение чисел в десятичной системе: {result}')
        print('-------------------')
        print(f'Произведение чисел в двоичной системе: ' + multi_list)
        return multi_list
    else:
        print(error)
        